# An iMODFLOW model from synthetic data

This notebook illustrates how to setup a simple two-dimensional groundwater model using synthetic data for conductivity and boundary conditions. We assume you have some familiarity with iMODFLOW files and model concepts. This notebook shows a very basic workflow:
1. Generate some data
2. Store these data in `xarray.DataArray`s
3. Inspect the data using `matplotlib` and `xarray`'s built-in plotting functionality
4. Mutate some data inside a `DataArray`
5. Store the `DataArray`s in `OrderedDict` and use the `imod` package to write files so iMODFLOW can run it
6. After running the model (outside of the notebook), load the results
7. Finally, plot the results.

Basic equations and parameter values have been taken from these two papers: 
* Toth, 1963, A Theoretical Analysis of Groundwater Flow in Small Drainage Basins
* Xiao-Wei Jiang, Li Wan, Xu-Sheng Wang, Shemin Ge, and Jie Liu, 2008, Effect of exponential decay in hydraulic conductivity with depth on regional groundwater flow

In [ ]:
from collections import OrderedDict

import imod
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

%matplotlib inline

The head of the phreatic water table is given by:

$z_x(x) = z_0 + x \tan\alpha + \frac{a}{\cos(\alpha)} \sin(\frac{bx}{\cos \alpha})$ <div style="text-align: right"> (Toth, 1963) </div>

Conductivity decreases exponentially with depth, according to:

$k(z) = k_0 \exp[-A (z_s - z)]$ <div style="text-align: right"> (Jiang et al., 2009) </div>

As Python functions, this becomes:

In [ ]:
def phreatic_head(z0, x, a, alpha, b):
    """Synthetic ground surface, a la Toth 1963"""
    return (z0 + x * np.tan(alpha) + a / np.cos(alpha) 
            * np.sin((b * x) / (np.cos(alpha))))


def conductivity(k0, z, A):
    """Exponentially decaying conductivity"""
    return k0 * np.exp(-A * (1000.0 - z))

## Generate the phreatic boundary condition

We start by defining the parameters for the `phreatic_head` function:

In [ ]:
z0 = 1000.0
x = np.arange(0.0, 6000.0, 10.0) + 5.0
a = 15.0
b = np.pi / 750.0
alpha = 0.02

Next, we create an `xarray.DataArray` to hold the phreatic head.

In [ ]:
head_top = xr.DataArray(
    data=phreatic_head(z0, x, a, alpha, b),
    coords={"x": x},
    dims=("x")
)

Let's have a look at whether that worked out, by plotting the result:

In [ ]:
fig = plt.figure(figsize=(10, 5))
head_top.plot()

## Generate conductivity

We're going to setup the IBOUND DataArray first. We can use it as a template afterwards to generate the other arrays, like conducitivies and starting heads.

In [ ]:
ncol = x.size
nrow = 1
nlay = 100
coords = {"layer": np.arange(1, 101), "y": [5.0], "x": x}
dims = ("layer", "y", "x")

bnd = xr.DataArray(
    data=np.full((nlay, nrow, ncol), 1.0),
    coords=coords,
    dims=dims,
)

In [ ]:
k0 = 1.0 # m/d
A = 0.001 # decay parameter
z = np.arange(0.0, 1000.0, 10.0) + 5.0

k_z = xr.DataArray(
    data=conductivity(k0, z, A),
    coords={"z": z},
    dims=("z"),
)

Let's inspect the result again:

In [ ]:
fig = plt.figure(figsize=(13, 10))
k_z.plot(y="z")
plt.title("Conductivity (m/d)")
plt.gca().invert_yaxis()
plt.xlabel("x")

iMODFLOW works with discrete layers, rather than elevation. Additionally, the top layer is the first rather than the last. Hence, we use the `layer` coordinate from the `bnd` array, flip it around, and then rename the `z` coordinate.

Although `bnd` is a three-dimensional array, and `k_z` is a one-dimensional array, `xarray` can multiply using automatic broadcasting. Here, it means that `k_z` will be expanded to three dimensions to match `bnd`. `xarray` is able to do this because the `layer` dimension is present in both DataArrays, with identical values.

In [ ]:
k_z.coords["z"] = bnd.coords["layer"].values[::-1]
k_z = k_z.rename({"z": "layer"})
kh = bnd.copy() * k_z

## Generate the iMODFLOW model files

Finally, we initialize an `OrderedDict` to hold the DataArrays and use `imod` to write the model.

In [ ]:
model = OrderedDict()
model["bnd"] = bnd
# constant head in the first layer
model["bnd"].sel(layer=1)[...] = -1.0
model["kdw"] = kh * 10.0
model["vcw"] = 10.0 / kh
model["shd"] = bnd * head_top
# iMODFLOW won't run with just these packages, so we add dummy recharge.
model["rch"] = xr.full_like(bnd.sel(layer=1), 0.0)

In [ ]:
imod.write("toth", model)

The `.write` function has written all necessary IDF files, and generated a runfile. 

Before running the model, you might want to edit the runfile for `kdw` and `vcw` so iMODFLOW will output flow budget files.

## Load the results

`imod` makes it very use to load the results of all 100 layers. It takes a [globpath](https://docs.python.org/3/library/glob.html), so we can load all layers in a single statement.

Here, we use the `*` symbol to denote a wildcard in the path to load all IDF files in the `results/head/` folder.

In [ ]:
head = imod.idf.load("toth/results/head/*.idf")
# Take a look at the DataArray:
head

In [ ]:
fig = plt.figure(figsize=(10, 6))
head.isel(y=0).plot()
plt.gca().invert_yaxis()

## Look at streamlines

`xarray` won't give us streamlines right out of the box. We'll use `matplotlib`'s streamplot functionality instead, by providing it the necessary (two-dimensional) `numpy` arrays. 

Once again, we can easily load the flow results using the `load` function; we select the first (and only) row using `.isel` to get a 2D array; we call `.values` to get a `numpy` array.

In [ ]:
vz = imod.idf.load("toth/results/bdgflf/*.idf").isel(y=0).values[:]
vx = imod.idf.load("toth/results/bdgfrf/*.idf").isel(y=0).values[:]

Now let's plot the phreatic head on top, and the flowlines underneath:

In [ ]:
f, (ax1, ax2) = plt.subplots(ncols=1, nrows=2, sharex=True, gridspec_kw={'height_ratios': [0.2, 0.8]}, figsize=(15, 5))
ax1.plot(head_top["x"], head_top.values, color="k")
ax1.set_ylabel("head(m)", size=16)
ax2.streamplot(x, z[:], vx, vz, arrowsize=2)
ax2.invert_yaxis()
ax2.set_ylabel("layer", size=16)
ax2.set_xlabel("x", size=16)
f.set_figheight(10.0)